# Análisis de Datos en Recursos Humanos Colsubsidio

Un alto porcentaje de las compañías tiende a establecer categorías en donde enmarcar a sus empleados, clientes, etc. Estas categorías permiten diferenciar fácilmente a las personas situadas en ellas, facilitando su identificación, generalidades del grupo, características en común, etc. En ocasiones ubicar a una persona en una categoría puede ser difícil, ya que puede aplicar a más de una, esto obliga a que muchas veces este procedimiento se haga manual, siendo largo y tortuoso. 

### Objetivo:

Automatizar la clasificación de empleados de Colsubsidio según sus características en las categorías previamente establecidas por la compañía.


In [1]:
#All imports

import warnings
import pandas as pd
import numpy as np
import pydotplus

from datetime import datetime, date, time, timedelta
from IPython.core.pylabtools import figsize

from Preprocessing import Preprocess
from Discretizing import Discretize
from DecisionTree import DecisionTree

figsize(14, 5)

warnings.filterwarnings('ignore')

file = 'files/database.csv'
dataset = pd.read_csv(file, index_col=False, header=0, delimiter="\t")

print dataset

print "La longitud del conjunto de datos es de", dataset.shape[0], "filas y", dataset.shape[1], "columnas."

C:\Users\Jonathan\Anaconda2\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


          ID                                    CARGO FECHA INICIO POSESION  \
0       5607    MEDICO CIRUJANO PEDIATRA HOSPITALARIO            29/05/2013   
1       5611    MEDICO CIRUJANO PEDIATRA HOSPITALARIO            29/05/2013   
2       5643    MEDICO CIRUJANO PEDIATRA HOSPITALARIO            29/05/2013   
3       6075    MEDICO CIRUJANO PEDIATRA HOSPITALARIO            29/05/2013   
4       8440    MEDICO CIRUJANO PEDIATRA HOSPITALARIO            29/05/2013   
5       3513        MEDICO ESPECIALISTA HEMATONCOLOGO            15/07/2010   
6       4626        MEDICO ESPECIALISTA HEMATONCOLOGO            19/08/2008   
7       3977        MEDICO ESPECIALISTA HEMATONCOLOGO            15/07/2010   
8      11486     GERENTE PROYECTO ALIMENTOS Y BEBIDAS            28/09/2016   
9       7357                        AUXILIAR MERCADEO             1/01/1960   
10      5727                         AUXILIAR EMPAQUE            17/01/2008   
11      3039                        AUXILIAR COCINA 

## Preprocesamiento.

#### Modulo de preprocesamiento.
Este modulo es usado para reemplazar valores faltantes usando la media de los datos. También se borran las variables que consideramos no eran de utilidad ya que no aportaban información o eran redundantes con otras.


In [ ]:
preprocess = Preprocess( dataset )
dataset = preprocess.preprocessFile()

PREPROCESSING MODULE 


Finding Minimum Wages
Replacing missing data in FECHA INICIO POSESION
Replacing missing data in PRIMERA ALTA
Replacing missing data in FECHA EXPIRACION CONTRATO
Deleting variables tat we consider useless ['ID', 'FECHA DE NACIMIENTO', 'SALARIO', 'SUELDO TEXTO', 'HORAS SEMANALES', 'HORAS DIARIAS', 'FIN']
La longitud del conjunto de datos es de 12394 filas y 28 columnas.


#### Discretización


In [ ]:
plotting = True #False to disable plotting
discretize = Discretize( dataset, True )
dataset = discretize.discretizeFile()

DISCRETIZE MODULE 


EDAD DEL EMPLEADO


## Agrupación

## Clasificación

Para la parte de clasificación se implementaron los clasificadoress de Naive Bayes, Árboles de decisión, Redes neuronales y Support Vector Machine. Veremos el accuracy de cada uno y una tabla comparativa entre ellos.

#### Naive Bayes

#### Árboles de decisión

In [ ]:
decisionTree = DecisionTree( dataset )
decisionTree.run()

#### Redes Neuronales

#### Support Vector Machine
